<a href="https://colab.research.google.com/github/fredlarochelle/Classify/blob/master/notebooks/0.1_fl_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import re
import random
import string
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
def get_proxy():
  response_proxy = requests.get("https://free-proxy-list.net/")

  soup = bs(response_proxy.content, "html.parser")

  tds = soup.find("tbody").find_all("tr")

  proxies = []

  for io in tds:
    proxies.append(f"{io.find_all('td')[0].text}:{io.find_all('td')[1].text}")

  return proxies

In [4]:
def get_headers(typed=None):
  if not typed:
    headers = {
      "authority": "twitter.com",
      "sec-ch-ua": '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
      "x-twitter-client-language": "en",
      "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
      "accept-encoding": "gzip, deflate, br",
      "upgrade-insecure-requests": "1",
      "sec-ch-ua-platform": 'Windows"',
      "sec-ch-ua-mobile": "?0",
      "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36",
    }
  else:
    headers = {
      'x-csrf-token': ''.join(random.choices(string.ascii_letters + string.digits, k=32)),
      'authorization': "Bearer " + "AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
      'content-type': "application/json",
      'referer': "https://twitter.com/AmitabhJha3",
      "authority": "twitter.com",
      "sec-ch-ua": '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
      "x-twitter-client-language": "en",
      "upgrade-insecure-requests": "1",
      "sec-ch-ua-platform": 'Windows"',
      "sec-ch-ua-mobile": "?0",
      "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36",
      "x-guest-token": typed
    }

  return headers

In [5]:
def get_guest_token():
  try:
    response = requests.get("https://twitter.com/i/trends", headers=get_headers(), proxies=proxy)

    guest_token = re.findall(
      'document\.cookie = decodeURIComponent\("gt=(.*?); Max-Age=10800; Domain=\.twitter\.com; Path=/; Secure"\);',
      response.text)
    
    try:
      return guest_token[0]
    except IndexError:
      raise ValueError("Guest token couldn't be found. Aborting...")
      
  except:
    raise ValueError("Response didn't work...")

In [8]:
i = 0

while True:
  try:
    proxy = {"http": random.choice(get_proxy())}
    guest_headers = get_headers(get_guest_token())
    i += 1
  except:
    continue
  else:
    print("Proxy and guest headers generated on try #", i)
    break

Proxy and guest headers generated on try # 1


In [9]:
# TODO:
# Twitter sometimes returns the same cursor as requested and no results even though there are more results.
# When this happens, retry the same cursor up to the retries setting.
#
# check if end cursor?
#
# Check "timeline" for more stuff, ads tags, order, ...

def parser(response):
  response_json = response.json()

  cursor = []

  for i in response_json['timeline']['instructions'][0]['addEntries']['entries']:
    if 'operation' in i['content']: # check to make it uniform with the second way of doing it
      if i['content']['operation']['cursor']['cursorType'] == "Bottom":
        cursor.append(i['content']['operation']['cursor']['value'])

  if len(cursor) == 0:
    for i in response_json['timeline']['instructions']:
      for key in i.keys():
        if key == "replaceEntry":
          if i['replaceEntry']['entry']['content']['operation']['cursor']['cursorType'] == "Bottom":
            cursor.append(i['replaceEntry']['entry']['content']['operation']['cursor']['value'])
            break

  data = []

  for element in response_json['globalObjects']['tweets']:
    data.append([response_json['globalObjects']['tweets'][element]["created_at"], response_json['globalObjects']['tweets'][element]["full_text"], response_json['globalObjects']['tweets'][element]["id"], response_json['globalObjects']['tweets'][element]["user_id"]])

  return data, cursor

In [10]:
search_url = "https://twitter.com/i/api/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweet=true&q={}&count=20&query_source=typeahead_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel%2CvoiceInfo"

def search(query, pages=1):
  session = requests.Session()
  session.headers = guest_headers
  session.proxies = proxy

  url = f"{search_url}&tweet_search_mode=live"
  r = session.get(url.format(query))

  tweets_data, cursor = parser(r)

  for i in range(pages):
    url = f"{search_url}&tweet_search_mode=live&cursor={cursor[0]}"
    #r = session.get(url.format(query), headers=guest_headers, proxies=proxy)
    r = session.get(url.format(query))
    tweets_data_temp, cursor = parser(r)

    tweets_data.extend(tweets_data_temp)

  return pd.DataFrame(tweets_data, columns=["created_at", "full_text", "id", "user_id"])

In [12]:
%time test = search('canada', pages=100)

CPU times: user 817 ms, sys: 36.5 ms, total: 853 ms
Wall time: 33 s


In [13]:
test

,created_at,full_text,id,user_id
0,Thu Dec 23 19:05:21 +0000 2021,"@Marcus4Georgia Wishing you luck from Canada, ...",1474093789027479562,1348019973776617476
1,Thu Dec 23 18:29:18 +0000 2021,"COVID-19 cases are spiking, hospitals are runn...",1474084716160733199,51476542
2,Thu Dec 23 19:05:16 +0000 2021,Canada could learn a lot from them.\nJapan’s s...,1474093769129701397,909278625219923968
3,Thu Dec 23 19:05:11 +0000 2021,90 في المائة من الإصابات الجديدة بكورونا من مت...,1474093749571489809,1343749659861213185
4,Thu Dec 23 19:05:11 +0000 2021,"@PhilPower19 Nah, there's certainly a nation m...",1474093747868925952,822178106924470272
...,...,...,...,...
2144,Thu Dec 23 18:23:04 +0000 2021,@kaki385 Hostage diplomacy obvious! Don't forg...,1474083147512008705,3581784381
2145,Thu Dec 23 18:23:18 +0000 2021,Water quality #FreshWater #GreatLakes #Michiga...,1474083208442662923,55704450
2146,Thu Dec 23 18:23:09 +0000 2021,"Ladies and gentlemen, the Minister of Transpor...",1474083170609860617,1105377223
2147,Thu Dec 23 11:13:55 +0000 2021,As that big bully pulls awful stunts on #Lithu...,1473975149678723072,976995099170062338


In [23]:
type(test['full_text'][0])

str

In [29]:
test.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  2149 non-null   object
 1   full_text   2149 non-null   object
 2   id          2149 non-null   int64 
 3   user_id     2149 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 899.8 KB


In [14]:
2149/33

65.12121212121212